In [1]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
from datetime import datetime
import json

region = 'euw1'

lol = LolWatcher(key)

In [2]:
me = lol.summoner.by_name(region,'Katiounette')
puuid = me['puuid']

In [3]:
matchs = lol.match.matchlist_by_puuid(region,puuid, count = 100)
i = 100
while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
    matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
    i = i + 100

In [17]:
matchData = pd.DataFrame({'playerID' : [],'matchID' : [], 'duration (s)' : [], 'start' : [], 'end' : [], 'championName' : [], 'championId' : [], 'pings' : [], 'ePings' : []})
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings


In [18]:
for j,i in enumerate(matchs):
    p = lol.match.by_id(region,i)
    for player in p['info']['participants']:
        if player['puuid'] == puuid:
            cName = player['championName']
            cId = player['championId']
            ePing = player['enemyMissingPings']
            pings = player["allInPings"] + player["assistMePings"] + player["baitPings"] + player["basicPings"] + player["commandPings"] + player["dangerPings"] + player["getBackPings"] + player["holdPings"] + player["needVisionPings"] + player["onMyWayPings"] + player["pushPings"] + player['visionClearedPings'] + player['enemyMissingPings'] + player['enemyVisionPings']
    matchData.loc[j,:] = [puuid,p['metadata']['matchId'],p['info']['gameDuration'], p['info']['gameStartTimestamp'], p['info']['gameEndTimestamp'], cName, cId, pings, ePing]
    if j == 100:
        break

In [4]:
from leagueDataParser import parseLeagueData

In [5]:
data = parseLeagueData(lol.match.by_id(region,matchs[1]), puuid)
pd.DataFrame(data, index=[0])

,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,...,primaryVar23,primaryVar33,secondaryPerk0,secondaryVar10,secondaryVar20,secondaryVar30,secondaryPerk1,secondaryVar11,secondaryVar21,secondaryVar31
0,1682535752822,1229,1682537155884,6377599750,CLASSIC,teambuilder-match-6377599750,1682535926790,MATCHED_GAME,13.8.505.1214,11,...,0,0,8139,435,0,0,8135,210,3,0


In [19]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6367571487,1857.0,1.681925e+12,1.681927e+12,MissFortune,21.0,4.0,3.0
1,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6365174837,1035.0,1.681760e+12,1.681761e+12,Illaoi,420.0,1.0,0.0
2,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6365123438,994.0,1.681759e+12,1.681760e+12,MissFortune,21.0,4.0,1.0
3,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363787945,1707.0,1.681672e+12,1.681674e+12,Kaisa,145.0,11.0,6.0
4,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363740926,915.0,1.681670e+12,1.681671e+12,Tristana,18.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
96,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301473433,1871.0,1.677927e+12,1.677929e+12,MissFortune,21.0,1.0,1.0
97,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301469276,110.0,1.677926e+12,1.677927e+12,MissFortune,21.0,0.0,0.0
98,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301449002,1237.0,1.677925e+12,1.677926e+12,Tristana,18.0,1.0,0.0
99,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301057974,2622.0,1.677882e+12,1.677885e+12,MissFortune,21.0,5.0,3.0


In [20]:
def getHourCat(i):
    h = matchData.loc[i,'startHour']
    if h >= 6 and h < 12:
        matchData.loc[i,'startHourCat'] = 'Morning'
    elif h >= 12 and h < 17:
        matchData.loc[i,'startHourCat'] = "Afternoon"
    elif h >= 17 and h < 21:
        matchData.loc[i,'startHourCat'] = "Evening"
    else :
        matchData.loc[i,'startHourCat'] = "Night"

In [21]:
for i in range(matchData.shape[0]):
    matchData.loc[i,'start'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'start']/1000))
    matchData.loc[i,'end'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'end']/1000))
    matchData.loc[i,'weekDay'] = matchData.loc[i,'start'].dayofweek
    matchData.loc[i,'weekDayName'] = matchData.loc[i,'start'].day_name()
    matchData.loc[i,'startHour'] = matchData.loc[i,'start'].hour

matchData.apply(lambda x : getHourCat(x.name), axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
96     None
97     None
98     None
99     None
100    None
Length: 101, dtype: object

In [22]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings,weekDay,weekDayName,startHour,startHourCat
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6367571487,1857.0,2023-04-19 19:29:53.700000,2023-04-19 20:00:51.414000,MissFortune,21.0,4.0,3.0,2.0,Wednesday,19.0,Evening
1,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6365174837,1035.0,2023-04-17 21:35:06.514000,2023-04-17 21:52:21.759000,Illaoi,420.0,1.0,0.0,0.0,Monday,21.0,Night
2,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6365123438,994.0,2023-04-17 21:14:43.723000,2023-04-17 21:31:17.856000,MissFortune,21.0,4.0,1.0,0.0,Monday,21.0,Night
3,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363787945,1707.0,2023-04-16 21:05:23.379000,2023-04-16 21:33:51.144000,Kaisa,145.0,11.0,6.0,6.0,Sunday,21.0,Night
4,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363740926,915.0,2023-04-16 20:41:07.486000,2023-04-16 20:56:23.379000,Tristana,18.0,2.0,2.0,6.0,Sunday,20.0,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301473433,1871.0,2023-03-04 11:46:37.151000,2023-03-04 12:17:48.126000,MissFortune,21.0,1.0,1.0,5.0,Saturday,11.0,Morning
97,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301469276,110.0,2023-03-04 11:39:57.290000,2023-03-04 11:41:47.472000,MissFortune,21.0,0.0,0.0,5.0,Saturday,11.0,Morning
98,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301449002,1237.0,2023-03-04 11:13:34.770000,2023-03-04 11:34:12.052000,Tristana,18.0,1.0,0.0,5.0,Saturday,11.0,Morning
99,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301057974,2622.0,2023-03-03 23:18:03.961000,2023-03-04 00:01:46.195000,MissFortune,21.0,5.0,3.0,4.0,Friday,23.0,Night


In [87]:
matchDataFinal = pd.DataFrame({'mondayTime' : [], 'tuesdayTime' : [], 'wednesdayTime' : [], 'thursdayTime' : [], 'fridayTime' : [], 'saturdayTime' : [], 'sundayTime' : [], 'morningTime' : [], 'afternoonTime' : [], 'eveningTime' : [], 'nightTime' : [], 'mondayCount' : [], 'tuesdayCount' : [], 'wednesdayCount' : [], 'thursdayCount' : [], 'fridayCount' : [], 'saturdayCount' : [], 'sundayCount' : [], 'morningCount' : [], 'afternoonCount' : [], 'eveningCount' : [], 'nightCount' : []})
matchDataFinal

,mondayTime,tuesdayTime,wednesdayTime,thursdayTime,fridayTime,saturdayTime,sundayTime,morningTime,afternoonTime,eveningTime,...,tuesdayCount,wednesdayCount,thursdayCount,fridayCount,saturdayCount,sundayCount,morningCount,afternoonCount,eveningCount,nightCount


In [116]:
matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1)

weekDay,0.0,1.0,2.0,3.0,4.0,5.0,6.0
playerID,,,,,,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,9,12,10,19,12,18,19


In [117]:
matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1)

weekDay,0.0,1.0,2.0,3.0,4.0,5.0,6.0
playerID,,,,,,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,15420.0,17745.0,15944.0,26807.0,20484.0,27926.0,27429.0


In [118]:
matchData.loc[:,['matchID', 'playerID', "startHourCat"]].groupby(['playerID','startHourCat']).count().unstack(level=1).droplevel(level = 0,axis = 1)

startHourCat,Evening,Night
playerID,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,37,64


In [120]:
matchDay = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1)

try:
    matchDataFinal.loc[:,"mondayTime"] = matchDay.loc[:,0]
except:
    matchDataFinal.loc[:,"mondayTime"] = 0
matchDataFinal.loc[:,"tuesdayTime"] = matchDay.loc[:,1]
matchDataFinal.loc[:,"wednesdayTime"] = matchDay.loc[:,2]
matchDataFinal.loc[:,"thursdayTime"] = matchDay.loc[:,3]
matchDataFinal.loc[:,"fridayTime"] = matchDay.loc[:,4]
matchDataFinal.loc[:,"saturdayTime"] = matchDay.loc[:,5]
matchDataFinal.loc[:,"sundayTime"] = matchDay.loc[:,6]

matchDay = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1)

matchDataFinal.loc[:,"mondayCount"] = matchDay.loc[:,0]
matchDataFinal.loc[:,"tuesdayCount"] = matchDay.loc[:,1]
matchDataFinal.loc[:,"wednesdayCount"] = matchDay.loc[:,2]
matchDataFinal.loc[:,"thursdayCount"] = matchDay.loc[:,3]
matchDataFinal.loc[:,"fridayCount"] = matchDay.loc[:,4]
matchDataFinal.loc[:,"saturdayCount"] = matchDay.loc[:,5]
matchDataFinal.loc[:,"sundayCount"] = matchDay.loc[:,6]

In [121]:
matchDataFinal

,mondayTime,tuesdayTime,wednesdayTime,thursdayTime,fridayTime,saturdayTime,sundayTime,morningTime,afternoonTime,eveningTime,...,tuesdayCount,wednesdayCount,thursdayCount,fridayCount,saturdayCount,sundayCount,morningCount,afternoonCount,eveningCount,nightCount
playerID,,,,,,,,,,,,,,,,,,,,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,15420.0,17745.0,15944.0,26807.0,20484.0,27926.0,27429.0,NaN,NaN,NaN,...,12,10,19,12,18,19,NaN,NaN,NaN,NaN
